In [59]:
from pathlib import Path
import glob

import numpy as np
from matplotlib import pyplot as plt

import os
import sys

from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QVBoxLayout, QDialog
from PyQt5.QtWidgets import *

import cv2


In [60]:
# %reset

In [44]:
from pathlib import Path
import glob

import numpy as np
from matplotlib import pyplot as plt

import os
import sys

from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QVBoxLayout, QDialog
from PyQt5.QtWidgets import *

import cv2

cv2.__version__


class ImageLabeler:
    
    def __init__(self):
        app = QApplication.instance()

        if app is None:
            app = QApplication(sys.argv)
        else:
            print('QApplication instance already exists: %s' % str(app))

        self.engine = LabelerEngine()
        self.gui = GUI(app, self.engine)
        
        self.engine.gui = self.gui
        
        app.exec_()


class LabelerEngine:

    SELECT_IMG = 1
    DESELECT_IMG = -1
    
    class Config:
    
        # path to the dataset containing subdirectories with images
        dataset_path = '/disks/data/datasets/selfie_project/selfie_dataset_versions/100k_copy'

        confirmed_dir = 'confirmed_'
        # sub directory where the to-be inspected reside
        #class_label = 'selfie'

        # flag deciding whether to show the selected image in a separate window
        #show_selected_images = False

        # image montage parameters
        MONTAGE_COL_COUNT = 5
        MONTAGE_ROW_COUNT = 5

        # default size of individual images in the montage (height, width)
        IMSIZE = (150, 150)

        IMGRID_SHAPE = (MONTAGE_ROW_COUNT, MONTAGE_COL_COUNT)
        IMGRID_SIZE = np.prod(IMGRID_SHAPE)

        class_names = [fname for dirpath,fname in [os.path.split(x) for x in glob.glob(os.path.join(dataset_path,'*'))] if not fname.startswith('confirmed_') ]

        class_label = class_names[0]# if len(Config.class_names)>0 else None        

        
        @classmethod
        def setup_paths(cls, class_name = None):

            if not class_name is None:
                cls.class_label = class_name 

            # new directories under 'dataset_path' where filtered images will be moved to 
            cls.moved_img_class_no = cls.class_label + '_no'
            cls.moved_img_class_yes = cls.class_label + '_yes'

            #cls.dataset_path = Path(cls.dataset_path_str)
            cls.img_path = os.path.join(cls.dataset_path, cls.class_label)

            # full paths of directories where selected and unselected images will be moved to
            cls.newdir_yes = os.path.join(cls.dataset_path, os.path.join(cls.confirmed_dir,cls.moved_img_class_yes) )
            cls.newdir_no = os.path.join(cls.dataset_path, os.path.join(cls.confirmed_dir,cls.moved_img_class_no) )

            # create directories for the moved images
            os.makedirs(cls.newdir_yes, exist_ok = True)
            os.makedirs(cls.newdir_no, exist_ok = True)
    
    
    def __init__(self):
        
        self.config = LabelerEngine.Config
        self.config.setup_paths(class_name = None) # default class: the first one
        
        self.gui = None  # updated later
        
        self.load_image_names(self.config.class_label)
               
        #self.generateNextPanelImage()
    
    def getClasses(self):
        
        return self.config.class_names
    
    
    def getNextPanelImage(self, container_height, container_width):
        
        # process selected (and unselected) images
        #process_selected_imgs(self.selected_imgs_names, self.im_fnames_montage)
        
        self.current_montage_start_idx = self.current_montage_start_idx + 1

        i = self.montage_img_start_indices[self.current_montage_start_idx]


        
        # generate indices for the list of all images        
        img_count = len(self.img_files)

        # indices of the images in current montage    
        imgs_idx_montage = slice(i, min(i+self.config.IMGRID_SIZE, img_count) )
        
        # filenames of the images in the montage
        self.im_fnames_montage = self.img_files[imgs_idx_montage]

        im_height = container_height//self.config.IMGRID_SHAPE[0]
        im_width = container_width//self.config.IMGRID_SHAPE[1]
        self.config.IMSIZE = (im_height, im_width) 
        
        # build the montage image        
        self.image_montage = self._create_montage(self.im_fnames_montage, self.config.IMGRID_SHAPE, self.config.IMSIZE)

        
        return self.image_montage
    
    def getCurrentPanelImage(self):
        return self.image_montage
    
    def load_image_names(self, class_name):
        
        # list containing the filenames of the selected images
        self.selected_imgs_names = []
        self.selected_imgs_coords = []
        
        print('loading images from disk')
        self.config.setup_paths(class_name = class_name)
        
        # read in list of image files
        self.img_files = glob.glob(os.path.join(self.config.img_path,'*.jpg'))

        # generate indices for the list of all images        
        im_indices = list(range(len(self.img_files)))
        
        self.montage_img_start_indices = im_indices[::self.config.IMGRID_SIZE]
        self.current_montage_start_idx = -1
        
        
    
    def _create_montage(self, imglist, montage_shape, img_sz):
        '''
        creates montage from a list of image filename, montage shape, and per-image size.

        params:
            imglist (str): list of filepaths of the images
            montage_shape (tuple): contains the dimensions (row_count, col_count) of the montage
            img_sz (tuple): contains the dimensions (height, width [, depth) of each image in the montage

        returns:
            numpy ndarray containing the montage of images.
        '''
        
        # create an empty canvas for the montage
        canvas = np.zeros( (img_sz[0]*montage_shape[0], img_sz[1]*montage_shape[1], 3), np.uint8)

        # fill-up the montage row-by-row
        for icol in range(montage_shape[1]):
            for irow in range(montage_shape[0]):
                # x-y coords to linear index 
                img_idx = irow*montage_shape[1] + icol

                # read and resize the image; on failure use a red placeholder
                try:
                    img_fname = imglist[img_idx]
                    img = cv2.resize(cv2.imread(img_fname),(img_sz[1],img_sz[0]))
                except Exception as e:
                    img = np.zeros((*img_sz[:2],3), np.uint8)
                    img[::-1] = 255
                    print('file read error',img_fname,e)

                # populate the canvas with the image
                canvas[irow*img_sz[0]: (irow+1)*img_sz[0],
                      icol*img_sz[1]: (icol+1)*img_sz[1],:] = img

        # return the montage        
        return canvas
        
    
    def process_selected_imgs(self):
        '''
        Move selected and unselected images to other, separate directories

        params:
            selected_imgs (list): list of selected images' paths
            images (list): list of all images' paths from the currently shown montage

        returns:
            Nothing
        '''

        # convert list to sets to eliminate duplicates
        selected_imgs = set(self.selected_imgs_names)
        # find the list of unselected images
        unselected_imgs = list(set(self.im_fnames_montage)-set(selected_imgs))

        # generate paths of the moved (selected and unselected) images
        newpaths_yes = [os.path.join(self.config.newdir_yes, fname) for fname in [ fname for path, fname in [os.path.split(filepath) for filepath in unselected_imgs ] ] ]
        newpaths_no = [os.path.join(self.config.newdir_no, fname) for fname in [ fname for path, fname in [os.path.split(filepath) for filepath in selected_imgs ] ] ]

        # move the selected and unselected images
        [os.rename( fr_file, to_file) for fr_file, to_file in zip(unselected_imgs, newpaths_yes)]
        [os.rename( fr_file, to_file) for fr_file, to_file in zip(selected_imgs, newpaths_no)]

        # list containing the filenames of the selected images
        self.selected_imgs_names = []
        self.selected_imgs_coords = []
 
    
    def act_on_image_at(self, click_x, click_y, action):
        
        # get montage image-space coordinates of the selected image
        def get_rectangle_corners(top_left, im_size, line_width):
            return ( (top_left[0]*im_size[1] + line_width//2, 
                      top_left[1]*im_size[0] + line_width//2),
                     ((top_left[0]+1)*im_size[1] - line_width//2, 
                      (top_left[1]+1)*im_size[0]- line_width//2) 
                   )
    
        def draw_border(im_coords, line_width, color):
                # montage image-space coordinates
                corners = get_rectangle_corners(im_coords, self.config.IMSIZE, line_width)

                # index of the selected image in the montage
                im_idx = im_coords[1]*self.config.IMGRID_SHAPE[1] + im_coords[0]

                # the file name of the selected image
                im_name = self.im_fnames_montage[im_idx]
                #print(im_idx, im_name)

                # draw a rectangular border around the selected image
                cv2.rectangle(self.image_montage, corners[0], corners[1], color, line_width )            

                return im_name

        
        
        # width of the rectangle border to be drawn around the selected image
        line_width = 6
            
        if(action == self.SELECT_IMG):
            # grid-space (zero-indexed) coordinates of the selected image
            im_coords = (click_x//self.config.IMSIZE[1], click_y//self.config.IMSIZE[0])

            print(click_x, click_y)
            print(self.config.IMSIZE)
            print(im_coords)
            
            self.selected_imgs_coords.append(im_coords)

            im_name = draw_border(im_coords, line_width,  (0,0,255))

            self.selected_imgs_names.append(im_name)
                
        elif(action == self.DESELECT_IMG):
            if len(self.selected_imgs_names) > 0:

                # remove the image file name from no-move queue
                im_name = self.selected_imgs_names.pop()

                # retrieve the coordinate of the previous selection
                latest_img_coords = self.selected_imgs_coords.pop()

                # draw a green border around the de-selected image
                draw_border(latest_img_coords, line_width, (0,255,0))   
        
        
class MontageLabel(QLabel):
    
    def __init__(self, parent = None):
        super(MontageLabel, self).__init__(parent)
        self.parent = parent
        
    def mouseReleaseEvent(self, ev: QMouseEvent):
        pos = ev.pos()
        
        print(self.pixmap().height())
        print(self.pixmap().width() )
        
        self._ActiveButton = ev.button()

        if self._ActiveButton == Qt.LeftButton:
            print("Left")
            self.parent.engine.act_on_image_at(pos.x(), pos.y(), self.parent.engine.SELECT_IMG)
            self.parent.redraw_img()
        elif self._ActiveButton == Qt.RightButton:
            print("Right")            
            self.parent.engine.act_on_image_at(pos.x(), pos.y(), self.parent.engine.DESELECT_IMG)
            self.parent.redraw_img()
            
class GUI(QWidget):
        
    def __init__(self, app, engine, parent=None):
        
        super(GUI, self).__init__(parent)

        self.engine = engine
        self.app = app
        
        screen = app.primaryScreen()
        size = screen.size()
        self.size = (size.width(), size.height())
        rect = screen.availableGeometry()
        self.visible_sz = (rect.width(), rect.height())

        # container for the montage image
        layout = QGridLayout(self)
        self.label = MontageLabel(self)#QLabel(self)
        self.label.setStyleSheet("background-color: white; inset grey; min-height: 200px;")
        self.label.setFrameShape(QFrame.Panel)
        self.label.setFrameShadow(QFrame.Sunken)
        
        class_names = engine.getClasses()
                
        vlayout = QVBoxLayout()
        
        self.comboBox = QComboBox(self)
        self.comboBox.addItem("")
        for class_name in class_names:
            self.comboBox.addItem(class_name)
            
        # make the first item non-selectable    
        model = self.comboBox.model()
        first_index = model.index(0, self.comboBox.modelColumn(), self.comboBox.rootModelIndex());
        model.itemFromIndex(first_index).setSelectable(False)   
        
        self.comboBox.activated[str].connect(self.class_combobox_selected)
                        
        vlayout.addWidget(self.comboBox)
        vlayout.addStretch(4)
        
        layout.addWidget(self.label,0,0)
        layout.addItem(vlayout,0,1)
        
        layout.setColumnStretch(0, 15)
        layout.setColumnStretch(1, 1)

        self.setLayout(layout)
                
        self.showMaximized()
        
        self.is_img_loaded = False
 
                                               
    def _np2pixmap(self, npimg):

        pilimg = cv2.cvtColor(npimg, cv2.COLOR_BGR2RGB)

        height, width, byteValue = npimg.shape
        byteValue = byteValue * width
        mQImage = QImage(pilimg, width, height, byteValue, QImage.Format_RGB888)

        pixmap = QPixmap(mQImage)
                                               
        return pixmap                                               
                                               
    def class_combobox_selected(self, text):
        print(text)
        
        self.engine.load_image_names((str(self.comboBox.currentText())))
        np_img = self.engine.getNextPanelImage(self.label.height(), self.label.width())        
        
        self.label.setPixmap(self._np2pixmap(np_img))
        self.label.setFocus()
        self.repaint()
        self.is_img_loaded = True

    def redraw_img(self):        
        np_img = self.engine.getCurrentPanelImage()        
        
        self.label.setPixmap(self._np2pixmap(np_img))
        self.label.setFocus()
        self.repaint()
        
#     def paintEvent(self, QPaintEvent):
#         painter = QPainter()
#         painter.begin(self)
#         painter.drawImage(0, 0, self.mQImage)
#         print(self.mQImage.height)
#         painter.end()

    def keyPressEvent(self, QKeyEvent):
        
        if(self.is_img_loaded and 'n' == QKeyEvent.text()):
            super(GUI, self).keyPressEvent(QKeyEvent)

            print('key pressed')
            self.engine.process_selected_imgs()
            np_img = self.engine.getNextPanelImage(self.label.height(), self.label.width())        
        
            self.label.setPixmap(self._np2pixmap(np_img))
            self.label.setFocus()
            self.repaint()
            
        
if __name__=="__main__":
    ImageLabeler()

QApplication instance already exists: <PyQt5.QtWidgets.QApplication object at 0x7f5f13fb5ee8>
loading images from disk
selfie
loading images from disk
1390
2340
Left
1734 221
(278, 468)
(3, 0)
1390
2340
Left
2128 682
(278, 468)
(4, 2)
1390
2340
Left
672 939
(278, 468)
(1, 3)
1390
2340
Left
216 977
(278, 468)
(0, 3)
1390
2340
Left
682 1291
(278, 468)
(1, 4)
1390
2340
Left
1646 1244
(278, 468)
(3, 4)
key pressed


In [10]:
Config.dataset_path_str

'/disks/data/datasets/selfie_project/selfie_dataset_versions/100k_copy/'

In [11]:
Config.IMGRID_SHAPE

(5, 5)

In [95]:
img = cv2.imread('/disks/data/datasets/selfie_project/selfie_dataset_versions/100k_copy/non_selfie/balkenende.jpg')

AttributeError: 'NoneType' object has no attribute 'shape'

In [16]:
Config.setup_config()

In [17]:
print(Config.newdir_yes)

/disks/data/datasets/selfie_project/selfie_dataset_versions/100k_copy/yes_selfie


In [66]:
class Config:
    
        # path to the dataset containing subdirectories with images
        dataset_path_str = '/disks/data/datasets/selfie_project/selfie_dataset_versions/100k_copy'

        # sub directory where the to-be inspected reside
        #class_label = 'selfie'

        # flag deciding whether to show the selected image in a separate window
        show_selected_images = False

        # image montage parameters
        MONTAGE_COL_COUNT = 5
        MONTAGE_ROW_COUNT = 5

        # size of individual images in the montage (width, height)
        IMSIZE = (150,150)

        IMGRID_SHAPE = (MONTAGE_COL_COUNT, MONTAGE_ROW_COUNT)
        IMGRID_SIZE = np.prod(IMGRID_SHAPE)

        class_names = [fname for dirpath,fname in [os.path.split(x) for x in glob.glob(os.path.join(dataset_path_str,'*'))] if not fname.startswith('yes_') and not fname.startswith('no_') ]

        class_label = class_names[0]# if len(Config.class_names)>0 else None        

        @staticmethod
        def setup_paths(class_name = None):

            if not class_name is None:
                Config.class_label = class_name 

            # new directories under 'dataset_path' where filtered images will be moved to 
            Config.moved_img_class_no = 'no_' + Config.class_label
            Config.moved_img_class_yes = 'yes_' + Config.class_label

            Config.dataset_path = Path(Config.dataset_path_str)
            Config.img_path = Config.dataset_path/Config.class_label

            # full paths of directories where selected and unselected images will be moved to
            Config.newdir_yes = Config.dataset_path/Config.moved_img_class_yes
            Config.newdir_no = Config.dataset_path/Config.moved_img_class_no

            # create directories for the moved images
            os.makedirs(Config.newdir_yes, exist_ok = True)
            os.makedirs(Config.newdir_no, exist_ok = True)

In [67]:
Config.setup_paths()

In [68]:
Config.moved_img_class_no

'no_non_selfie'